In [15]:
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
import rasterio
import math
import plotly.express as px
import plotly.offline as py 
import plotly.graph_objs as go 
from plotly.figure_factory import create_table
import georasters as gr
from pyproj import Proj, transform
import pandas as pd
from plotly.graph_objects import Densitymapbox

In [16]:
# Reads in TIFF files and converts to numpy arrays

veg_dist_date = Image.open('VEG_DIST_DATE.tif')
dist_date = np.array(veg_dist_date)

veg_anom_max = Image.open('VEG_ANOM_MAX.tif')
anom_max = np.array(veg_anom_max)

veg_dist_status = Image.open('VEG_DIST_STATUS.tif')
dist_status = np.array(veg_dist_status)

M, N = dist_date.shape

In [17]:
# Creates dataframes for each TIFF file

dist_status = gr.from_file('VEG_DIST_STATUS.tif')
dist_status = dist_status.to_pandas()
dist_status = dist_status.rename(columns={'x': 'Longitude', 'y': 'Latitude'})

anom_max = gr.from_file('VEG_ANOM_MAX.tif')
anom_max = anom_max.to_pandas()
anom_max = anom_max.rename(columns={'x': 'Longitude', 'y': 'Latitude'})

dist_date = gr.from_file('VEG_DIST_DATE.tif')
dist_date = dist_date.to_pandas()
dist_date = dist_date.rename(columns={'x': 'Longitude', 'y': 'Latitude'})

In [18]:
# Filters out low vegetation loss areas and dates before (approx.) April 06

extent_df = dist_status.rename(columns = {'value': 'status value'})
extent_df['max anom value'] = anom_max['value']
extent_df['date value'] = dist_date['value']
extent_df = extent_df[['row', 'col', 'Latitude', 'Longitude', 'status value', 'max anom value', 'date value']]
extent_df = extent_df[extent_df['max anom value'] >= 10]
extent_df = extent_df[extent_df['date value'] >= 466]

In [19]:
# Converts latitude and longitude to "normal" coordinates

inProj = Proj(init='epsg:32613')
outProj = Proj(init='epsg:4326')


extent_df['Longitude'], extent_df['Latitude'] = transform(inProj, outProj, extent_df['Longitude'], extent_df['Latitude'])

/Users/rdhillon/opt/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

/Users/rdhillon/opt/anaconda3/lib/python3.9/site-packages/pyproj/crs/crs.py:131: FutureWarning:

'+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6

/var/folders/54/n31gxnv93j3g7p_9gvm8jtbm0000gq/T/ipykernel_30747/1454032722.py:7: DeprecationWarning:

This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1



In [20]:
# Displays dataframe so far

extent_df

,row,col,Latitude,Longitude,status value,max anom value,date value
17,0,17,36.139789,-106.106257,1.0,15,507
28,0,28,36.139823,-106.102590,2.0,28,481
333,0,333,36.140716,-106.000904,2.0,18,476
338,0,338,36.140730,-105.999237,2.0,26,483
360,0,360,36.140791,-105.991902,1.0,16,498
...,...,...,...,...,...,...,...
13386241,3659,3335,35.155161,-104.999890,0.0,47,503
13386255,3659,3349,35.155160,-104.995279,0.0,50,503
13386257,3659,3351,35.155160,-104.994620,0.0,14,503
13386546,3659,3640,35.155119,-104.899428,1.0,23,478


In [21]:
# Initializes empty dataframe

column_names = ['Latitude', 'Longitude', 'status value', 'day']
large_extent_df = pd.DataFrame(columns=column_names)
large_extent_df

,Latitude,Longitude,status value,day


In [ ]:
# Fills empty dataframe with data from every 5 days

large_extent = []
for x in range(466, 514, 5):
    final_extent_df = extent_df
    mask = np.empty(dist_date.shape)
    final_extent_df['status value'].mask(final_extent_df['date value'] <= x, 0, inplace=True)
    final_extent_df = final_extent_df.drop(columns=['max anom value', 'date value', 'row', 'col'])
    final_extent_df = final_extent_df[final_extent_df['status value'] != 0]
    final_extent_df['day'] = x
    
    large_extent.append(final_extent_df)

large_extent_df = pd.concat(large_extent)

In [ ]:
large_extent_df

In [ ]:
# Plots wildfire extent with time slider (in scatterplot form, but we want it in raster form)

fig = px.density_mapbox(large_extent_df, lat=large_extent_df['Latitude'], lon=large_extent_df['Longitude'], 
                        animation_frame='day', radius=1, 
                        center=dict(lat=35.6, lon=-105.45), zoom=10, mapbox_style='stamen-terrain', 
                        range_color=(1,4))
fig.update_layout(autosize=False, width=1500, height = 1500)
fig["layout"].pop("updatemenus")
fig.show()